In [ ]:
# First download and extract certificates to this directory
# https://drive.google.com/file/d/1kjnwSdBmoUrK4croKmP5ELmWzXRnuxMz/view?usp=drive_link

import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import re

def load_consts(file):
    with open(file, "r") as f:
        data = list(filter(lambda x: x != "" and x[0] != "#", map(lambda x: x.strip(), f.readlines())))

    data = list(map(lambda x: x.replace("TPM2_ALG_", ""), data))

    return data

ALGS = load_consts("algs.txt")

def candidate_fitness(candidate):
    lname = candidate['name'].lower()
    ldata = candidate['report_data'].lower() + candidate["target_data"].lower()

    match = {
        "name": False,
        "data": 0,
        "skip": False,
    }

    if "tpm" in lname or "trusted platform module" in lname or "trusted plattform module" in lname:
        match["name"] = True

    match["data"] += ldata.count("tpm")
    match["data"] += ldata.count("trusted platform module")

    if "tivoli" in lname:
        match['skip'] = True
    if "trunk pack module" in lname:
        match['skip'] = True
    if "lexmark" in lname:
        match['skip'] = True
    if "atmel" in lname:
        match['skip'] = True

    return match

# Revisions
def select_rev(matches, full_data):
    matches = set(matches)
    tpm12 = 0
    tpm20 = 0.0
    for match in matches:
        if match in ("62", "85", "94", "103", "116"):
            tpm12 = max(tpm12, int(match))

        if match in ("0.96", "0.99", "1.16", "1.38", "1.59"):
            tpm20 = max(tpm20, float(match))


    if tpm12 != 0 and tpm20 != 0.0:
        tpm20_matches = len(re.findall("TPM\s*2\.?0", full_data))
        tpm12_matches = len(re.findall("TPM\s*1\.?2", full_data))
        if tpm20_matches >= tpm12_matches:
            tpm12 = 0
        else:
            tpm20 = 0.0

    if tpm12 != 0:
        return str(tpm12)
    elif tpm20 != 0.0:
        return str(tpm20)
    return None

def get_rev(candidate):
    data = candidate["report_data"] + candidate['target_data']
    rev = list(map(lambda x: x[2], re.findall(r"r(ev)?(ision)?\s*(\d+\.?\d+)", data, re.IGNORECASE)))
    rev += list(map(lambda x: x, re.findall(r"v(\d+\.\d+)", data, re.IGNORECASE)))
    rev = list(filter(lambda x: x in ("62", "85", "94", "103", "116", "0.96", "0.99", "1.16", "1.38", "1.59"), rev))


    if rev:
        rev = select_rev(rev, data)
    else:
        match candidate["dgst"]:
            case "8885e51cb974ed23" | "6465914348251eac" | "962944088b147de1" | "6529877d9799822a" | "ac1257069b2f4afd" | "3976c9e492193315" | "f11c7004acecc201" | "373ec112d46d6c97" | "63b0155fe5f82cce" | "dc99a69a88c91874":
                rev = "1.59"
            case "6dde54b8c6d592ef" | "9652ec3b28f1be86" | "5cadc087c21ab8db":
                rev = "1.38"
            case "732271484650d27a" | "1d1df0fb541e49b8" | "808de32aa819821d" | "edfe491576c461c4" | "e0283686d26420d9" | "b21be5dd6fdec83b" | "94fb86d31446e531" | "9ae8bef18938d2df" | "3281f5d91ee7433e" | "131927c002782b5d"  | "d5b5fab21d68556e" | "9dfbae98257df8db" | "7768e375a15e957a" | "665727ccadb38d2d" | "c50f690cc8d00710":
                rev = "1.16"
            case "e9ed6f2d52911d5c" | "0167c92c0d8c8b47" | "d5c7e2e456748263" | "c0c67d95f05b7f74" | "213f6505c6c2a611" | "d376881f5bd00764" | "9c55b07c46330c66" | "d398b29d3dbbb9bf":
                rev = "116"
            case "2ff8ceac4a6ca519":
                rev = "94"
            case _:
                rev = "None"

    return rev


def load_cc(fname):
    with open(fname, "r") as f:
        data = json.load(f)

    out = []

    for cert in data["certs"]:
        candidate = {
            "name": cert["name"],
            "version": {},
            "api": {},
            "packages": {},
            "year": int(cert["not_valid_before"].split("-")[0]),
            "dgst": cert["dgst"],
            "std": "cc",
            "algs": {key: 0 for key in ALGS}
        }

        try:
            with open(f"{candidate['std']}/report/txt/{candidate['dgst']}.txt", "rb") as f:
                report_data = f.read()
        except:
            report_data = b""

        try:
            with open(f"{candidate['std']}/target/txt/{candidate['dgst']}.txt", "rb") as f:
                target_data = f.read()
        except:
            target_data = b""

        candidate["report_data"] = report_data.decode("utf-8", "ignore")
        candidate["target_data"] = target_data.decode("utf-8", "ignore")

        for alg in ALGS:
            candidate["algs"][alg] += candidate["report_data"].count(alg)
            candidate["algs"][alg] += candidate["target_data"].count(alg)

        candidate['fitness'] = candidate_fitness(candidate)

        if not candidate['fitness']['skip'] and (candidate['fitness']["name"] or candidate['fitness']["data"] > 110):
            candidate['rev'] = get_rev(candidate)
            try:
                candidate['level'] = str(cert["pdf_data"]["st_keywords"]["cc_security_level"]),
            except:
                candidate['level'] = ""
            out.append(candidate)

    return out

def load_fips(fname):
    with open(fname, "r") as f:
        data = json.load(f)

    out = []

    for cert in data["certs"]:
        if not cert["web_data"] or not cert["dgst"] or not cert["web_data"]["module_name"]:
            continue

        candidate = {
            "name": cert["web_data"]["module_name"],
            "version": {},
            "api": {},
            "packages": {},
            "year": int(cert["web_data"]["validation_history"][0]["date"].split("-")[0]),
            "dgst": cert["dgst"],
            "std": "fips",
            "algs": {key: 0 for key in ALGS}
        }

        try:
            with open(f"{candidate['std']}/report/txt/{candidate['dgst']}.txt", "rb") as f:
                report_data = f.read()
        except:
            report_data = b""

        try:
            with open(f"{candidate['std']}/target/txt/{candidate['dgst']}.txt", "rb") as f:
                target_data = f.read()
        except:
            target_data = b""

        candidate["report_data"] = report_data.decode("utf-8", "ignore")
        candidate["target_data"] = target_data.decode("utf-8", "ignore")

        for alg in ALGS:
            candidate["algs"][alg] += candidate["report_data"].count(alg)
            candidate["algs"][alg] += candidate["target_data"].count(alg)

        candidate['fitness'] = candidate_fitness(candidate)

        if not candidate['fitness']['skip'] and (candidate['fitness']["name"] or candidate['fitness']["data"] > 110):
            candidate['rev'] = get_rev(candidate)
            try:
                candidate['level'] = str(cert["pdf_data"]["keywords"]["fips_security_level"]),
            except:
                candidate['level'] = ""
            out.append(candidate)

    return out


In [ ]:
cc_data = load_cc("cc.json")
fips_data = load_fips("fips.json")
data = cc_data + fips_data
data.sort(key=lambda x: x['fitness']['data'], reverse=True)

In [ ]:
from collections import defaultdict
filtered = list(filter(lambda x: "." in x["rev"], data))

# count CC certs
print("TPM2 CC certs:", len(list(filter(lambda x: x["std"] == "cc", filtered))))
print("TPM2 FIPS certs:", len(list(filter(lambda x: x["std"] == "fips", filtered))))

vendors = defaultdict(list)
for cert in filtered:
    vendor = "Unknown"
    if "Atmel" in cert["name"]:
        vendor = "Atmel"
    elif "Virtual TPM" in cert["name"]:
        vendor = "Microsoft"
    elif "ST33" in cert["name"]:
        vendor = "STM"
    elif "Infineon" in cert["name"]:
        vendor = "Infineon"
    elif "SLB" in cert["name"]:
        vendor = "Infineon"
    elif "Nuvoton" in cert["name"]:
        vendor = "Nuvoton"
    elif "NPCT" in cert["name"]:
        vendor = "Nuvoton"
    elif "FB5C85D" in cert["name"]:
        vendor = "Nuvoton"
    elif "SLM" in cert["name"]:
        vendor = "Infineon"
    vendors[f"{vendor}-{cert['rev']}"].append(cert)

In [ ]:
tmp = {vendor: {i: 0 for i in range(2015, 2024)} for vendor in vendors}
for vendor in vendors:
    for cert in vendors[vendor]:
        year = cert["year"]
        if year not in tmp[vendor]:
            tmp[vendor][year] = 0
        tmp[vendor][year] += 1

tmp

df = pd.DataFrame.from_dict(tmp, orient="columns")
df.sort_index(inplace=True)

import pickle
pickle.dump((tmp, df), open("certs.pkl", "wb"))

In [ ]:
import pickle
tmp, df = pickle.load(open("certs.pkl", "rb"))

fig, ax = plt.subplots(figsize=(10, 4))
bottom = np.zeros(len(df.index))

cmap = plt.get_cmap("Paired")
colors = dict(zip(["Infineon", "Microsoft", "Nuvoton", "STM"], ["#d81b60", "#04ab8f", "#ffc107", (30/255, 136/255, 229/255, 1.0)]))
hatches = dict(zip(["1.16", "1.38", "1.59"], ["/", "\\", "x"]))

for vendor_rev, counts in sorted(tmp.items(), key=lambda x: (x[0].split("-")[0], -float(x[0].split("-")[1])), reverse=True):
    vendor, rev = vendor_rev.split("-")
    ax.bar(counts.keys(), counts.values(), bottom=bottom, color=colors[vendor], hatch=hatches[rev], label=vendor, edgecolor="black")#, align="edge", width=1)
    bottom += df[vendor_rev]

ax.set_xticks(list(df.index))

color_handles = [mpatches.Patch(facecolor=colors[vendor], label=vendor) for vendor in colors]
hatch_handles = [mpatches.Patch(facecolor="white", edgecolor="black", hatch=hatches[rev], label=f"rev {rev}") for rev in hatches]
ax.legend(handles=color_handles + hatch_handles, loc="upper left", ncol=2)

plt.savefig("certs.pdf", dpi=600, bbox_inches="tight")
plt.show()